In [1]:
import os 
import re
import matplotlib.pyplot as plt
import rmgpy.chemkin
import numpy as np
import cantera as ct
import pandas as pd
%matplotlib inline



In [2]:
def load_chemkin_file(path): 
    """ 
    Load Chemkin file and extract the reactions and species.
    Path should be the path to the species folder.
    """
    
    full_path = os.path.join(path,'chemkin')
    chemkin_path = os.path.join(full_path,'copies', 'copy_chem_annotated.inp')
   # chemkin_path = os.path.join(full_path,'chem_annotated.inp')
    dictionary_path = os.path.join(full_path,'species_dictionary.txt')
    transport_path = os.path.join(full_path,'tran.dat')

    species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(chemkin_path, dictionary_path=dictionary_path, transport_path=transport_path)

    return species_list, reaction_list

In [4]:
############# load the chemkin file and the cantera dataframe ################


chemkin_path = '/work/westgroup/nora/Code/projects/Burgess_Comments/2_BTP_optimization/models/edited_RMG_with_BROH/edited_RMG_with_BROH_777.inp'
dictionary_path = '/work/westgroup/nora/Code/projects/Burgess_Comments/2_BTP_optimization/models/RMG_with_BROH/chemkin/species_dictionary.txt'
transport_path = '/work/westgroup/nora/Code/projects/Burgess_Comments/2_BTP_optimization/models/RMG_with_BROH/chemkin/tran.dat'

species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(chemkin_path, dictionary_path=dictionary_path, transport_path=transport_path)


#df = pd.read_csv('./2_BTP_sensitivities.csv')
df = pd.read_csv('/work/westgroup/nora/Code/projects/Burgess_Comments/2_BTP_optimization/models/edited_RMG_with_BROH/edited_RMG_with_BROH_777.cti')

ParserError: Error tokenizing data. C error: Expected 4 fields in line 2987, saw 5


## Sensitivity Plot

In [5]:
#we want to focus on the top ___ most sensitive reactions 
n = len(reaction_list)

In [ ]:
############ plotting stuff

fig, ax = plt.subplots()

plt.xlabel('Sensitivity')
plt.title(f"Sensitivity for 2_BTP")

############# iterate over rows of csv file. 
    
for index,row in df.head(n).iterrows(): #Only iterate over the first n rows
    
        
    sensitivity, ct_equation, = row['sensitivity'], row['cantera equation']
    ax.barh(ct_equation, sensitivity, color='b', align='center')


## Analysis of Reaction Families

In [13]:

i=0 # counter, for assertion error later 
reaction_family_data = []  #to save reaction family data 



for index,row in df.head(n).iterrows(): #Only iterate over the first n rows
    
        
    ct_index, sensitivity, ct_equation, ct_products, ct_reactants = row['k_s'], row['sensitivity'], row['cantera equation'], eval(row['cantera products']), eval(row['cantera reactants'])
   
    #list of all products in each reaction 
    list_of_products = [(k+',')*int(v) for k,v in ct_products.items()]   #use ',' as a marker for distinction between different species
    products = set(list(filter(None, (''.join(list_of_products)).split(',')))) #now make all the products into a list. If two of the same species are listed as reactants, they should appear as two entries

    #list of all reactants in each reaction 
    list_of_reactants = [(k+',')*int(v) for k,v in ct_reactants.items()]   #use ',' as a marker for distinction between different species
    reactants= set(list(filter(None, (''.join(list_of_reactants)).split(',')))) #now make all the products into a list. If two of the same species are listed as reactants, they should appear as two entries

    
    #iterate through rmg reactions and find the match to the cantera reaction
    for rmg_rxn in reaction_list: 
            rmg_reactants = set([str(i.to_chemkin()) for i in rmg_rxn.reactants])
            rmg_products = set([str(i.to_chemkin()) for i in rmg_rxn.products])
            
            if (reactants==rmg_reactants) and (products==rmg_products) and ('M' in ct_equation) and ('M' in str(rmg_rxn)):
                reaction_family_data.append(rmg_rxn.family)
                #print(ct_index,'appended')
                i += 1 #for assertion error
                break
            if (reactants==rmg_reactants) and (products==rmg_products) and ('M' not in ct_equation) and ('M' not in str(rmg_rxn)):
                try: 
                    reaction_family_data.append(rmg_rxn.family)
                    #print(ct_index,'appended')
                    i += 1 #for assertion error
                    break
                except AttributeError:
                    reaction_family_data.append('PDepReaction')
                    #print(ct_index, 'appended')
                    i += 1 #for assertion error
                    break
            if (reactants==rmg_reactants) and (products==rmg_products)and ('M' in ct_equation) and ('M' not in str(rmg_rxn)) :
                reaction_family_data.append('PDepReaction')
                i += 1 #for assertion error
                break

print(i,n)
assert i==n, "Not all equations in Cantera sensitivity analysis were matched!"




############ organize data into a second dataframe
data = {
    'cantera equations': df['cantera equation'][:n],
    'sensitivities': df['sensitivity'][:n],
    'reaction family': reaction_family_data,
}


df2 = pd.DataFrame(data)

df2

############## STYLE IT UP
halogen_families = [
    'Cl_Abstraction',
    'F_Abstraction',
    'Br_Abstraction',
    'XY_Addition_MultipleBond',
    '1,2_XY_interchange',
    'halocarbene_recombination',
    'halocarbene_recombination_double',
    'halocarbene_CO_dimerization',
    'XY_elimination_hydroxyl',
    'intra_halogen_migration',
    'Disproportionation-Y',
]

target_family = ['halocarbene_recombination']

def highlight_rows(x):
    if x['reaction family'] in target_family:
        return['background-color: pink']*3
    if x['reaction family'] in halogen_families:
        return['background-color: yellow']*3
    else:
        return['background-color: white']*3
display(df2.style.apply(highlight_rows, axis = 1))


1474 1474
